In [1]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sector_etfs = ['XLE', 'XLF', 'XLK', 'XLI', 'XLB', 'XLV', 'XLY', 'XLP', 'XLU', 'XLRE'] # Add XLC?
sector_twin_etfs = [
    'XLF', 'VFH',   # Financials
    'XLK', 'VGT',   # Tech
    'XLY', 'VCR',   # Consumer Discretionary
    'XLP', 'VDC',   # Consumer Staples
    'XLV', 'VHT',   # Healthcare
    'XLE', 'VDE',   # Energy
    'XLI', 'VIS',   # Industrials
    'XLB', 'VAW',   # Materials
    'XLU', 'VPU',   # Utilities
    'XLRE', 'VNQ',  # Real Estate
    'VOX',   # Comms without XLC
]
dividend_etfs = ['VYM', 'HDV', 'SCHD', 'DVY']
bond_etfs = ['SHY', 'VGSH', 'IEF', 'TLH', 'TLT', 'SPTL']
industry_etfs = ['KBE', 'XOP', 'IHF', 'SOXX', 'ITA', 'IBB']
thematic_etfs = ['ARKK', 'ARKW', 'ROBO', 'BOTZ', 'TAN', 'ICLN', 'LIT'] # Add BATT?
international_etfs = ['EFA', 'VEA', 'IEMG', 'EEM', 'EWJ', 'EWU', 'EWG', 'FXI', 'EWZ']

etf_categories = {
    'sector': sector_etfs,
    'sector_twins': sector_twin_etfs,
    'bond': bond_etfs,
    'thematic': thematic_etfs,
}


flattened_etfs = [etf for etfs in etf_categories.values() for etf in etfs]

prices = yf.download(flattened_etfs, start = '2010-01-01', end = '2019-12-31')['Close']

for etf in prices.columns:
    data = prices[etf]

    if data.empty:
        print(f"{etf} does not contain data")

# Pearson correlation

for category, etfs in etf_categories.items():
    desired_etfs = [etf for etf in etfs]

    category_prices = prices[desired_etfs]

    returns = category_prices.pct_change().dropna()

    correlation_matrix = returns.corr()
    correlation_matrix = correlation_matrix.rename_axis(None).rename_axis(None, axis = 1)
    correlation_matrix = correlation_matrix.stack().reset_index()
    correlation_matrix.columns = ['ETF', 'Pair', 'Correlation']

    correlation_matrix =  correlation_matrix[correlation_matrix['ETF'] != correlation_matrix['Pair']]
    correlation_matrix = correlation_matrix.sort_values(by = 'Correlation', ascending = False)
    correlation_matrix = correlation_matrix.drop_duplicates('Correlation')

    potential_pairs = correlation_matrix[correlation_matrix['Correlation'] > 0.8]

    print(f"Potential Correlated Pairs from {category}:")
    print(potential_pairs)

/var/folders/qd/xclxh05s0x9509nblm6z434w0000gp/T/ipykernel_56883/2726479297.py:44: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(flattened_etfs, start = '2010-01-01', end = '2019-12-31')['Close']
[*********************100%***********************]  34 of 34 completed


Potential Correlated Pairs from sector:
    ETF Pair  Correlation
26  XLK  XLY     0.847599
34  XLI  XLB     0.834225
31  XLI  XLF     0.803238
Potential Correlated Pairs from sector_twins:
      ETF Pair  Correlation
221   XLE  VDE     0.995401
373   VPU  XLU     0.993256
153   VDC  XLP     0.992817
21    VFH  XLF     0.992749
65    VGT  XLK     0.992254
109   VCR  XLY     0.991976
265   XLI  VIS     0.991684
177   XLV  VHT     0.988901
309   XLB  VAW     0.984735
397  XLRE  VNQ     0.931970
328   VAW  VIS     0.867210
108   VCR  VGT     0.853579
47    XLK  VCR     0.849242
267   XLI  VAW     0.848993
67    VGT  XLY     0.848931
86    XLY  XLK     0.847599
287   VIS  XLB     0.843716
306   XLB  XLI     0.834225
118   VCR  VIS     0.825232
34    VFH  VIS     0.823905
273   VIS  XLF     0.818597
97    XLY  VIS     0.809019
33    VFH  XLI     0.803381
12    XLF  XLI     0.803238
257   XLI  VCR     0.802500
Potential Correlated Pairs from bond:
     ETF  Pair  Correlation
29   TLT  SPTL  

In [2]:
# checking data point count
etf_data_counts = prices.notna().sum().sort_values()

etf_data_coverage = etf_data_counts.to_frame(name='Available Days')
etf_data_coverage.index.name = 'ETF'

print(etf_data_coverage.head(10)) 

      Available Days
ETF                 
BOTZ             830
XLRE            1064
ARKK            1299
ARKW            1322
ROBO            1558
LIT             2376
IEF             2515
ICLN            2515
SPTL            2515
TAN             2515


In [3]:
# cointegration

for category, etfs in etf_categories.items():

        candidate_pairs = []

        desired_etfs = [etf for etf in etfs]

        category_prices = prices[desired_etfs]

        for i in range(len(desired_etfs)):
                for j in range(i + 1, len(desired_etfs)):
                        candidate_pairs.append([desired_etfs[i], desired_etfs[j]])

        cointegrated_pairs = []

        for etf1, etf2 in candidate_pairs:

                df = category_prices[[etf1, etf2]].dropna()

                s_etf1 = df[etf1]
                s_etf2 = df[etf2]

                score, pvalue, _ = coint(s_etf1, s_etf2)

                if pvalue < 0.05:
                        cointegrated_pairs.append((etf1, etf2, pvalue))
                        
        cointegrated_pairs_df = pd.DataFrame(cointegrated_pairs, columns = ['ETF', 'Pair', 'P-Value'])
        cointegrated_pairs_df = cointegrated_pairs_df.sort_values('P-Value').reset_index(drop=True)

        print(f"Cointegration Test Results for {category}:")
        print(cointegrated_pairs_df)

Cointegration Test Results for sector:
   ETF  Pair   P-Value
0  XLI   XLB  0.003161
1  XLU  XLRE  0.009561
2  XLY   XLU  0.014643
3  XLF   XLI  0.016249
4  XLI   XLU  0.026338
5  XLB   XLU  0.039937
Cointegration Test Results for sector_twins:
    ETF  Pair   P-Value
0   XLP   VNQ  0.000403
1   VDC   VNQ  0.000858
2   VIS   XLB  0.002114
3   XLI   XLB  0.003161
4   XLU  XLRE  0.009561
5   VFH   XLI  0.013609
6   XLY   XLU  0.014643
7   XLF   XLI  0.016249
8   VPU  XLRE  0.019591
9   XLI   VPU  0.021824
10  VCR   VPU  0.024229
11  XLU   VNQ  0.024904
12  XLI   XLU  0.026338
13  VIS   VPU  0.027384
14  XLY   VPU  0.028307
15  VIS   XLU  0.031538
16  XLB   VPU  0.036763
17  XLF   VIS  0.038168
18  XLB   XLU  0.039937
19  VFH   VIS  0.043138
20  XLV   VNQ  0.043288
21  VPU   VNQ  0.047769
Cointegration Test Results for bond:
   ETF  Pair   P-Value
0  IEF   TLH  0.007771
1  TLH   TLT  0.008771
2  IEF   TLT  0.009283
3  IEF  SPTL  0.014453
4  TLH  SPTL  0.016573
Cointegration Test Results f

In [4]:
# rolling cointegration

window_size = 504
min_passes = 0.4
step = 30

for category, etfs in etf_categories.items():

    candidate_pairs = []
    rolling_cointegrated_pairs = []

    desired_etfs = [etf for etf in etfs]

    category_prices = prices[desired_etfs]

    for i in range(len(desired_etfs)):
        for j in range(i + 1, len(desired_etfs)):
                candidate_pairs.append([desired_etfs[i], desired_etfs[j]])

    for etf1, etf2 in candidate_pairs:
            
        df = category_prices[[etf1, etf2]].dropna()

        s_etf1 = df[etf1]
        s_etf2 = df[etf2]

        df = pd.concat([s_etf1, s_etf2], axis = 1)

        if len(df[etf1]) == 0:
            print(f"{etf1} does not have sufficient data")
            continue
        elif len(df[etf2]) == 0:
            print(f"{etf2} does not have sufficient data")
            continue
            
        series1 = df.iloc[:, 0]
        series2 = df.iloc[:, 1]

        cointegrated_windows = 0
        total_windows = 0

        for start in range(0, len(df) - window_size + 1, step):
            end = start + window_size

            window_s1 = series1.iloc[start:end]
            window_s2 = series2.iloc[start:end]
                
            score, pvalue, _ = coint(window_s1, window_s2)
            total_windows += 1
                
            if pvalue < 0.05:
                cointegrated_windows += 1

        if cointegrated_windows / total_windows >= min_passes:
            rolling_cointegrated_pairs.append({'ETF1': etf1, 'ETF2': etf2, 'Pass %': cointegrated_windows / total_windows})


    rolling_cointegrated_pairs_df = pd.DataFrame(rolling_cointegrated_pairs)

    if rolling_cointegrated_pairs_df.empty:
        print(f"{category} has no rolling cointegrated pairs.")
        continue
    else:
        rolling_cointegrated_pairs_df = rolling_cointegrated_pairs_df.sort_values('Pass %', ascending = False).reset_index(drop=True)
        print(f"Rolling Cointegration Test Results for {category}:")
        print(rolling_cointegrated_pairs_df)

KeyboardInterrupt: 

In [ ]:
# adf test / z-score spread

def zscore_calc(series):
    return (series - series.mean()) / series.std()

def adf_test(series):
    test_res = adfuller(series)
    return {'stat': test_res[0], 'p-value': test_res[1]}

def hedge_ratio_calc(series1, series2):
    x = sm.add_constant(series2)
    model = sm.OLS(series1, x).fit()

    return model.params.iloc[1]


for category, etfs in etf_categories.items():

    candidate_pairs = []
    rolling_cointegrated_pairs = []
    results = []

    desired_etfs = [etf for etf in etfs]

    category_prices = prices[desired_etfs]

    for i in range(len(desired_etfs)):
        for j in range(i + 1, len(desired_etfs)):
                candidate_pairs.append([desired_etfs[i], desired_etfs[j]])

    for etf1, etf2 in candidate_pairs:

        df = category_prices[[etf1, etf2]].dropna()

        s_etf1 = df[etf1]
        s_etf2 = df[etf2]

        hedge_ratio = hedge_ratio_calc(s_etf1, s_etf2)

        spread = s_etf1 - (hedge_ratio * s_etf2)

        zscore_spread = zscore_calc(spread)

        adf_res = adf_test(spread)

        results.append(
            {'ETF1': etf1,
            'ETF2': etf2,
            'adf_value': adf_res['stat'],
            'p-value': adf_res['p-value'],
            'mean': zscore_spread.mean(),
            'std': zscore_spread.std()}
        )

    results = pd.DataFrame(results)
    results = results.sort_values('p-value', ascending = True)
    results = results[results['p-value'] < 0.05]

    print(f"ADF Test Results for {category}:")
    print(results)

In [ ]:
# k-means / PCA

for category, etfs in etf_categories.items():
    desired_etfs = [etf for etf in etfs]

    category_prices = prices[desired_etfs]

    returns = category_prices.pct_change().dropna()

    scaler = StandardScaler()
    returns_scaled = scaler.fit_transform(returns.T)

    pca = PCA(n_components=2)
    pca_components = pca.fit_transform(returns_scaled)

    max_clusters = min(len(etfs) - 1, 10)

    best_k = 2
    best_score = -1

    for k in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(pca_components)
        score = silhouette_score(pca_components, labels)
        if score > best_score:
            best_score = score
            best_k = k

    kmeans = KMeans(n_clusters = best_k, random_state = 42)
    labels = kmeans.fit_predict(pca_components)

    cluster_df = pd.DataFrame({
        'ETF': returns.columns,
        'Cluster': labels,
        'PC1': pca_components[:, 0],
        'PC2': pca_components[:, 1]
    })

    print(f"Clusters for {category}:")
    print(cluster_df)

    sns.scatterplot(data=cluster_df, x='PC1', y='PC2', hue='Cluster')
    plt.title(f"Clusters for {category}")
    plt.show()

    cluster_df = cluster_df.sort_values(by='Cluster')